# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846833049048                   -0.70    4.5
  2   -7.852326126981       -2.26       -1.53    1.0
  3   -7.852646059163       -3.49       -2.52    3.2
  4   -7.852646677283       -6.21       -3.36    2.0
  5   -7.852646685876       -8.07       -4.79    1.2
  6   -7.852646686723       -9.07       -5.18    4.0
  7   -7.852646686729      -11.21       -6.09    1.2
  8   -7.852646686730      -12.45       -7.22    2.8
  9   -7.852646686730      -14.75       -7.44    2.0
 10   -7.852646686730   +    -Inf       -8.39    2.2
 11   -7.852646686730      -14.45       -8.72    1.0
 12   -7.852646686730   +  -14.45       -9.42    2.2
 13   -7.852646686730      -14.75      -10.88    2.0
 14   -7.852646686730   +    -Inf      -11.73    3.2
 15   -7.852646686730   +  -14.45      -12.13    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846864581043                   -0.70           4.8
  2   -7.852527762585       -2.25       -1.63   0.80    2.0
  3   -7.852637414090       -3.96       -2.71   0.80    1.0
  4   -7.852646424424       -5.05       -3.31   0.80    2.0
  5   -7.852646682284       -6.59       -4.13   0.80    2.0
  6   -7.852646686461       -8.38       -4.77   0.80    1.5
  7   -7.852646686722       -9.58       -5.82   0.80    1.5
  8   -7.852646686730      -11.13       -6.62   0.80    2.0
  9   -7.852646686730      -12.92       -7.81   0.80    1.8
 10   -7.852646686730      -14.75       -8.62   0.80    2.8
 11   -7.852646686730   +  -15.05       -8.56   0.80    2.0
 12   -7.852646686730      -14.57       -9.83   0.80    1.0
 13   -7.852646686730      -15.05      -10.72   0.80    2.2
 14   -7.852646686730   +  -14.57      -11.33   0.80    2.0
 15   -7.852646686730      -14.27      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.375395e+01     3.472760e+00
 * time: 0.4141199588775635
     1     1.345282e+00     1.778258e+00
 * time: 0.6162469387054443
     2    -1.536269e+00     2.131893e+00
 * time: 0.6397719383239746
     3    -3.827775e+00     1.829706e+00
 * time: 0.7522928714752197
     4    -5.058952e+00     1.595514e+00
 * time: 0.7857668399810791
     5    -6.646831e+00     1.036288e+00
 * time: 0.8192520141601562
     6    -7.332951e+00     5.384534e-01
 * time: 0.8527870178222656
     7    -7.610033e+00     4.313432e-01
 * time: 0.8759920597076416
     8    -7.750359e+00     1.210738e-01
 * time: 0.8991539478302002
     9    -7.793349e+00     1.383538e-01
 * time: 0.922576904296875
    10    -7.822164e+00     1.064798e-01
 * time: 0.9460320472717285
    11    -7.839552e+00     5.986846e-02
 * time: 0.9693589210510254
    12    -7.849165e+00     3.243035e-02
 * time: 0.9926860332489014
    13    -7.851341e+00     2.800450e-02
 * time: 1.01601004600

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846804920780                   -0.70    4.5
  2   -7.852285697773       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646683355                   -2.57
  2   -7.852646686730       -8.47       -5.21
  3   -7.852646686730   +  -14.75       -9.47
  4   -7.852646686730   +    -Inf      -15.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.178591894771385e-13
|ρ_newton - ρ_scfv| = 9.58070398855297e-14
|ρ_newton - ρ_dm|   = 7.189955606422062e-10
